In [ ]:
from stc_tools.client import StcTools
client = StcTools(paths=(
    '/ipns/standard-template-construct.org/data/nexus_books/',
    '/ipns/standard-template-construct.org/data/nexus_science/',
))
# Connects to IPFS and instantiate configured indices for searching
await client.setup()

In [ ]:
# Match search over books collection
await client.search([{
    "index_alias": "nexus_books",
    "query": {"query": {"match": {"value": "hemoglobin", "default_fields": ["title"], "field_boosts": {}}}},
    "collectors": [{"collector": {"top_docs": {"limit": 5}}}],
    "is_fieldnorms_scoring_enabled": False,
}])

In [ ]:
# Term search over science collection
await client.search([{
    "index_alias": "nexus_science",
    "query": {"query": {"term": {"field": "doi", "value": "10.1109/healthcom54947.2022.9982758"}}},
    "collectors": [{"collector": {"top_docs": {"limit": 1}}}],
    "is_fieldnorms_scoring_enabled": False,
}])

In [ ]:
# Complex query and count results too
SHOULD = 0
MUST = 1
MUST_NOT = 2

await client.search([{
    "index_alias": "nexus_science",
    "query": {"query": {"boolean": {"subqueries": [
        {"occur": SHOULD, "query": {"query": {"match": {"value": "hemoglobin", "default_fields": ["title"], "field_boosts": {}}}}},
        {"occur": SHOULD, "query": {"query": {"match": {"value": "fetal", "default_fields": ["title"], "field_boosts": {}}}}},
    ]}}},
    "collectors": [{"collector": {"top_docs": {"limit": 5}}}, {"collector": {"count": {}}}],
    "is_fieldnorms_scoring_enabled": False,
}])